In [ ]:
#default_exp renderer

In [ ]:
%pip install svgwrite

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.1.7/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#|export
from tetris.piece import Piece
from tetris.playfield import Playfield
import svgwrite
from IPython.display import SVG

class Renderer:
    def playfield_to_svg(self, pf: Playfield) -> SVG:
        A = pf.toarray()
        side_len = 10
        dwg = svgwrite.Drawing()
        dwg.viewbox(0,0,300,250)
        
        height = len(A)
        width = len(A[0])

        # border around entire playfield
        #dwg.add(dwg.rect((0, 0), (width*side_len, height*side_len), stroke="green"))

        for row in range(height):
            for col in range(len(A[0])):
                left_top = (col*side_len, (height-row)*side_len)
                width_height = (side_len, side_len)
                stroke = "#cccccc"
                fill = 'white'
                corner_radius = 0
                if A[row][col]:
                    stroke, fill = '#112543', '#d1eaf3'
                    corner_radius = 1
                dwg.add(dwg.rect(left_top, width_height, stroke=stroke, fill=fill, rx=corner_radius, ry=corner_radius))
        return SVG(dwg.tostring())
 
    def piece_to_svg(self, piece: Piece) -> SVG:
        side_len = 10
        dwg = svgwrite.Drawing()
        dwg.viewbox(0,0,300,50)

        A = piece.toarray()
        height = len(A)
        width = len(A[0])
        
        for row in range(height):
            for col in range(width):
                left_top = (col*side_len, (height-row)*side_len)
                width_height = (side_len, side_len)
                fill = 'blue' if A[row][col] else 'white'
                stroke = '#112543'
                dwg.add(dwg.rect(left_top, width_height, fill=fill, stroke=stroke))
        return SVG(dwg.tostring())